In [1]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import timm  # timm (PyTorch Image Models) library for accessing pre-trained models and utilities for image classification
import os  
from tqdm.notebook import tqdm  # tqdm for displaying progress bars in Jupyter notebooks
from torchvision import transforms as T, datasets
from helper import view_classify, show_image, show_grid, accuracy # Importing custom helper functions for visualization and calculating accuracy

In [2]:
# Configuration class to store hyperparameters and dataset information
class CFG:
    
    epochs = 20  # Number of training epochs
    lr = 0.001   # Learning rate for the optimizer
    batch_size = 16  # Batch size for training and validation

    model_name = 'DenseNetPneumoniaModel'  # Name of the model to be used, sourced from timm library
    img_size = 224  # Image size for input into the model

    # Paths for dataset directories
    DATA_DIR = "./chest_xray_data"  # Root directory for the dataset
    TEST = 'test'  # Subdirectory for test data
    TRAIN = 'train'  # Subdirectory for training data
    VAL = 'val'  # Subdirectory for validation data

# Setting the device for training/testing. Uses GPU if available, else CPU.
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Print statement to confirm the device being used
print("On which device we are on : {}".format(device))


On which device we are on : cuda:1


In [3]:
# Define transformations for the training dataset
train_transform = T.Compose([
    T.Resize(size=(CFG.img_size, CFG.img_size)),  # Resize images to the specified size in the CFG class
    T.RandomRotation(degrees=(-20, +20)),  # Apply random rotations between -20 and +20 degrees to augment the data
    T.ToTensor(),  # Convert images to PyTorch tensors
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize images using ImageNet's mean and std
])

# Define transformations for the validation dataset
valid_transform = T.Compose([
    T.Resize(size=(CFG.img_size, CFG.img_size)),  # Resize images to the specified size
    T.ToTensor(),  # Convert images to PyTorch tensors
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize images using the same mean and std
])

# Define transformations for the test dataset
test_transform = T.Compose([
    T.Resize(size=(CFG.img_size, CFG.img_size)),  # Resize images to the specified size
    T.ToTensor(),  # Convert images to PyTorch tensors
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize images using the same mean and std
])

In [4]:
# Constructing file paths for the training, validation, and test datasets using the CFG class
train_path = os.path.join(CFG.DATA_DIR, CFG.TRAIN)  # Path to the training dataset
valid_path = os.path.join(CFG.DATA_DIR, CFG.VAL)    # Path to the validation dataset
test_path = os.path.join(CFG.DATA_DIR, CFG.TEST)    # Path to the test dataset

print(train_path, valid_path, test_path)

# Loading datasets using ImageFolder, which expects data to be in a certain directory structure
trainset = datasets.ImageFolder(train_path, transform=train_transform)  # Training dataset with transformations
validset = datasets.ImageFolder(valid_path, transform=valid_transform)  # Validation dataset with transformations
testset = datasets.ImageFolder(test_path, transform=test_transform)     # Test dataset with transformations

./chest_xray_data\train ./chest_xray_data\val ./chest_xray_data\test


In [5]:
from torch.utils.data import DataLoader 

# Creating data loaders for the datasets
trainloader = DataLoader(trainset, batch_size=CFG.batch_size, shuffle=True)  # DataLoader for the training dataset
validloader = DataLoader(validset, batch_size=CFG.batch_size, shuffle=True)  # DataLoader for the validation dataset
testloader = DataLoader(testset, batch_size=CFG.batch_size, shuffle=False)    # DataLoader for the test dataset

In [9]:
import torch.nn as nn
model = timm.create_model(CFG.model_name, pretrained=False)  # Create a new instance of the model
model.classifier = nn.Sequential(
    nn.Linear(in_features=1920, out_features=625),  # First linear layer
    nn.ReLU(),  # ReLU activation function
    nn.Dropout(p=0.3),  # Dropout layer with a dropout probability of 0.3
    nn.Linear(in_features=625, out_features=256),  # Second linear layer
    nn.ReLU(),  # ReLU activation function
    nn.Linear(in_features=256, out_features=2)  # Final linear layer with 2 outputs for binary classification
    )

model.load_state_dict(torch.load('DenseNetPneumoniaModel.pth'))  # Load the saved state_dict
model.to(device)  # Move the model to the GPU)

c:\Users\jxs75\Documents\GitHub\Pneumonia Classification\Pneumonia-Classification\.venv\lib\site-packages\timm\models\_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


EfficientNet(
  (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          24, ep

In [10]:
# Trainer class for handling the training and validation of a neural network model for pneumonia detection
class PneumoniaTrainer():

    # Constructor for initializing the trainer with a loss function, optimizer, and scheduler
    def __init__(self, criterion=None, optimizer=None, schedular=None):
        self.criterion = criterion  # Loss function to use
        self.optimizer = optimizer  # Optimizer for adjusting model weights
        self.schedular = schedular  # Learning rate scheduler (if any)
    
    # Function to perform the training loop over all batches in the training data
    def train_batch_loop(self, model, trainloader):
        train_loss = 0.0  # Variable to accumulate loss over all batches
        train_acc = 0.0   # Variable to accumulate accuracy over all batches

        # Iterate over batches of images and labels in the training dataset
        for images, labels in tqdm(trainloader):
            
            # Move images and labels to the device (GPU or CPU)
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass: compute predicted outputs by passing images to the model
            logits = model(images)
            loss = self.criterion(logits, labels)  # Calculate the batch's loss

            # Backward pass and optimization
            self.optimizer.zero_grad()  # Clear the gradients of all optimized variables
            loss.backward()  # Perform backward pass to calculate gradients
            self.optimizer.step()  # Perform a single optimization step (parameter update)

            # Accumulate loss and accuracy for this batch
            train_loss += loss.item()
            train_acc += accuracy(logits, labels)

        # Calculate average loss and accuracy over all batches
        return train_loss / len(trainloader), train_acc / len(trainloader)
    
    # Function to perform the validation loop over all batches in the validation data
    def valid_batch_loop(self, model, validloader):
        
        valid_loss = 0.0  # Variable to accumulate loss over all batches
        valid_acc = 0.0   # Variable to accumulate accuracy over all batches

        # Iterate over batches of images and labels in the validation dataset
        for images, labels in tqdm(validloader):

            # Move images and labels to the device (GPU or CPU)
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass: compute predicted outputs by passing images to the model
            logits = model(images)
            loss = self.criterion(logits, labels)  # Calculate the batch's loss

            # Accumulate loss and accuracy for this batch
            valid_loss += loss.item()
            valid_acc += accuracy(logits, labels)

        # Calculate average loss and accuracy over all batches
        return valid_loss / len(validloader), valid_acc / len(validloader)
    
    # Function to train and validate the model
    def fit(self, model, trainloader, validloader, epochs):
        
        valid_min_loss = np.Inf  # Initialize minimum validation loss to infinity for tracking improvement

        # Training loop for the specified number of epochs
        for i in range(epochs):

            model.train()  # Set the model to training mode
            # Perform training over all batches and get average loss and accuracy
            avg_train_loss, avg_train_acc = self.train_batch_loop(model, trainloader)

            model.eval()  # Set the model to evaluation mode
            # Perform validation over all batches and get average loss and accuracy
            avg_valid_loss, avg_valid_acc = self.valid_batch_loop(model, validloader)

            # Check if validation loss has improved, if so, save the model
            if avg_valid_loss <= valid_min_loss:
                print("Valid_loss decreased {} --> {}".format(valid_min_loss, avg_valid_loss))
                torch.save(model.state_dict(), 'DenseNetPneumoniaModel.pth')  # Save the model's state_dict
                valid_min_loss = avg_valid_loss  # Update minimum validation loss

            # Print training and validation results for this epoch
            print("Epoch : {} Train Loss : {:.6f} Train Acc : {:.6f}".format(i+1, avg_train_loss, avg_train_acc))
            print("Epoch : {} Valid Loss : {:.6f} Valid Acc : {:.6f}".format(i+1, avg_valid_loss, avg_valid_acc))

In [11]:
model.train()  # Set the model to training mode
# Using Cross-Entropy Loss as the criterion for the classification task
criterion = nn.CrossEntropyLoss()

# Setting up the optimizer - Adam optimizer with a learning rate from CFG
optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr)

# Instantiating the PneumoniaTrainer class with the defined criterion and optimizer
trainer = PneumoniaTrainer(criterion, optimizer)

# Using the fit method of PneumoniaTrainer to start the training and validation process
trainer.fit(model, trainloader, validloader, epochs=CFG.epochs)

  0%|          | 0/326 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Valid_loss decreased inf --> 0.00969159509986639
Epoch : 1 Train Loss : 0.182147 Train Acc : 0.935391
Epoch : 1 Valid Loss : 0.009692 Valid Acc : 1.000000


  0%|          | 0/326 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch : 2 Train Loss : 0.098625 Train Acc : 0.965299
Epoch : 2 Valid Loss : 1.204638 Valid Acc : 0.687500


  0%|          | 0/326 [00:00<?, ?it/s]

In [ ]:
# Set the model to evaluation mode
model.eval()

# Evaluating the model on the test dataset
avg_test_loss, avg_test_acc = trainer.valid_batch_loop(model, testloader)

# Printing the test loss and accuracy
print("Test Loss : {}".format(avg_test_loss))  # Print average loss on the test dataset
print("Test Acc : {}".format(avg_test_acc))    # Print average accuracy on the test dataset

In [ ]:
torch.save(model.state_dict(), 'DenseNetPneumoniaModel.pth')

In [ ]:
del model  # Delete the model
torch.cuda.empty_cache()  # Clear GPU cache